# Описание проекта
Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

#### Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

---

## Описание данных

#### Признаки
 * **DateCrawled** — дата скачивания анкеты из базы
 * **VehicleType** — тип автомобильного кузова
 * **RegistrationYear** — год регистрации автомобиля
 * **Gearbox** — тип коробки передач
 * **Power** — мощность *(л.с.)*
 * **Model** — модель автомобиля
 * **Kilometer** — пробег *(км)*
 * **RegistrationMonth** — месяц регистрации автомобиля
 * **FuelType** — тип топлива
 * **Brand** — марка автомобиля
 * **NotRepaired** — была машина в ремонте или нет
 * **DateCreated** — дата создания анкеты
 * **NumberOfPictures** — количество фотографий автомобиля
 * **PostalCode** — почтовый индекс владельца анкеты *(пользователя)*
 * **LastSeen** — дата последней активности пользователя

#### Целевой признак
 * **Price** — цена *(евро)*

---
<a id='К_оглавлению'></a>

## Разделим данное исследоване на несколько этапов:

[Подготовка и предобработка](#Подготовка_и_предобработка)
* Импорт необходимых библиотек.
* Предобработка данных.
* Разделение таблицы на признаки и целефой признак.
* Кодировка данных и разделение на обучающую и валидационную выборки.

[Обученине моделей](#Обученине_моделей)
* **LightGBMRegressor**
* **XGBoostREgressor**
* **CatBoostRegressor**
* **RandomForestRegressor**

[Анализ моделей](#Анализ_моделей)

<a id='Подготовка_и_предобработка'></a>
# 1. Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from lightgbm import LGBMRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")


df = pd.read_csv('/datasets/autos.csv')
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Приведем названия столбцов к ниднему регистру.

In [2]:
df.columns = df.columns.str.lower()

В таблице присутствуют даты, посмотрим на типы признаков.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
datecrawled          354369 non-null object
price                354369 non-null int64
vehicletype          316879 non-null object
registrationyear     354369 non-null int64
gearbox              334536 non-null object
power                354369 non-null int64
model                334664 non-null object
kilometer            354369 non-null int64
registrationmonth    354369 non-null int64
fueltype             321474 non-null object
brand                354369 non-null object
notrepaired          283215 non-null object
datecreated          354369 non-null object
numberofpictures     354369 non-null int64
postalcode           354369 non-null int64
lastseen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


Даты имеют тип **object**. Приведем их к **datetime**.

In [4]:
for i in ('datecrawled', 'datecreated', 'lastseen', 'postalcode'):
    df[i] = pd.to_datetime(df[i], format='%Y-%m-%dT%H:%M:%S')

df.info()

Проверим *registrationyear* на аномальные значения

In [5]:
df['registrationyear'].sort_values().unique()

array([1000, 1001, 1039, 1111, 1200, 1234, 1253, 1255, 1300, 1400, 1500,
       1600, 1602, 1688, 1800, 1910, 1915, 1919, 1920, 1923, 1925, 1927,
       1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938,
       1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2066, 2200, 2222, 2290, 2500, 2800, 2900, 3000,
       3200, 3500, 3700, 3800, 4000, 4100, 4500, 4800, 5000, 5300, 5555,
       5600, 5900, 5911, 6000, 6500, 7000, 7100, 7500, 7800, 8000, 8200,
       8455, 8500, 8888, 9000, 9229, 9450, 9996, 99

Избавимся от аномалий.

In [6]:
df = df.loc[(df['registrationyear']>1900) & (df['registrationyear']<2020)]

In [7]:
df['registrationyear'].describe()

count    354198.000000
mean       2003.084789
std           7.536418
min        1910.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        2019.000000
Name: registrationyear, dtype: float64

Так же посмотрим столбцы цены и мощности.

In [8]:
df['price'].describe()

count    354198.000000
mean       4417.651314
std        4514.081022
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: price, dtype: float64

Ограничим минимальную цену.

In [9]:
df = df.loc[df['price'] > 50]

In [10]:
df['power'].describe()

count    341495.000000
mean        111.273275
std         188.192808
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: power, dtype: float64

Ограничим адекватными значениями лошадиных сил.

In [11]:
df = df.loc[(df['power'] > 30) & (df['power'] < 500)]

Теперь проверим на пропущенные значения.

In [12]:
df.isna().sum()

datecrawled              0
price                    0
vehicletype          20991
registrationyear         0
gearbox               5897
power                    0
model                12049
kilometer                0
registrationmonth        0
fueltype             19380
brand                    0
notrepaired          46439
datecreated              0
numberofpictures         0
postalcode               0
lastseen                 0
dtype: int64

Все пропущенные значения пнаходятся в столбцах с типом **object** заполним их занчениями *NaN*, что бы избежать ошибок в обучении.

In [13]:
df = df.fillna("NaN")

In [14]:
df.isna().sum()

datecrawled          0
price                0
vehicletype          0
registrationyear     0
gearbox              0
power                0
model                0
kilometer            0
registrationmonth    0
fueltype             0
brand                0
notrepaired          0
datecreated          0
numberofpictures     0
postalcode           0
lastseen             0
dtype: int64

Разделим данные на фичи и таргет, а так же уберем признаки, которые могут уложнить обучение модели.

In [15]:
features = df.drop(['price', 'datecrawled', 'datecreated', 'lastseen', 'postalcode'], axis=1)
target = df['price']

Закодируем категориальные данные.

In [16]:
#gd - get dummies
gd_features = pd.get_dummies(features, drop_first=True).copy()

Разобьем выборки для `catboost` и других моделей градиентного бустинга.

In [17]:
# c - coded
c_features_train, c_features_valid, c_target_train, c_target_valid = train_test_split(
gd_features, target, test_size=0.25, random_state=42)

features_train, features_valid, target_train, target_valid = train_test_split(
features, target, test_size=0.25, random_state=42)

Теперь масштабируем признаки.

In [18]:
numeric = ['registrationyear', 'power', 'kilometer', 'registrationmonth', 'numberofpictures']
scaler = StandardScaler()
scaler.fit(features_train[numeric])

c_features_train[numeric] = scaler.transform(c_features_train[numeric])
c_features_valid[numeric] = scaler.transform(c_features_valid[numeric])

## Вывод
Была проведена предобработка данных. Названия столбцов были приведены к единому формату. Признаки с датами переведены к нужному типу, удалены аномалии в *registrationyear*. Данные разделены на обучающие и валидацонные выборки.

[К оглавлению](#К_оглавлению)

<a id='Обученине_моделей'></a>

# 2. Обучение моделей

Для скорочти обучения я буду использовать `RandomizedSearchCV`, так как он работает быстрее `GridSearchCV`, хоть и менне точен.

---
**LightGBMRegressor**

Выберем параметры для модели и бучим ее.

In [19]:
lgbm_param_dist = {'num_leaves': [10, 30, 60],
              'n_estimators': [100, 120],
              'max_depth': [10, 25]}

In [20]:
lgbmr = LGBMRegressor(state=42)

In [21]:
lgbm_random_search = RandomizedSearchCV(estimator = lgbmr, param_distributions=lgbm_param_dist, cv=5, random_state=42)

In [22]:
%%time
lgbm_random_search.fit(c_features_train, c_target_train)

CPU times: user 10min 7s, sys: 20.9 s, total: 10min 28s
Wall time: 10min 35s


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           importance_type='split',
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=-1, num_leaves=31,
                                           objective=None, random_state=None,
                                           reg_alpha=0.0, reg_lambda=0.0,
                                           silent=True, state=42, subsample=1.0,
                                           subsample_for_bin=200000,
                      

In [23]:
lgbm_random_search.best_params_, lgbm_random_search.best_score_

({'num_leaves': 60, 'n_estimators': 120, 'max_depth': 25}, 0.8751531639268035)

In [24]:
%%time
mean_squared_error(c_target_valid, lgbm_random_search.predict(c_features_valid))**0.5

CPU times: user 1.89 s, sys: 71.7 ms, total: 1.96 s
Wall time: 1.9 s


1615.2310935785579

**XGBoostREgressor**

In [25]:
xgbr_param_dist = {'n_estimators': [20, 40, 60], 
              'max_depth': [10]}

In [26]:
xgbr = xgb.XGBRegressor(objective ='reg:linear', verbosity = 0,state=42)

In [27]:
xgbr_random_search = RandomizedSearchCV(estimator = xgbr, param_distributions=xgbr_param_dist, cv=2, random_state=42)

In [28]:
%%time
xgbr_random_search.fit(c_features_train, c_target_train)

CPU times: user 19min 24s, sys: 10.6 s, total: 19min 34s
Wall time: 19min 48s


RandomizedSearchCV(cv=2, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=0,
                                          reg_lambda=1, scale_pos_weight=1,
                                          seed=None, silent=None, state=42,
                     

In [29]:
xgbr_random_search.best_params_, xgbr_random_search.best_score_

({'n_estimators': 60, 'max_depth': 10}, 0.8748540608660615)

In [30]:
%%time
mean_squared_error(c_target_valid, xgbr_random_search.predict(c_features_valid))**0.5

CPU times: user 1.14 s, sys: 247 ms, total: 1.39 s
Wall time: 1.42 s


1589.3735055732147

**CatBoostRegressor**

In [31]:
cat_param = {
    'learning_rate': np.logspace(-3, 0),
    'iterations': [150, 200],
    'depth': [5, 10]}

In [32]:
cat_features = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']

In [33]:
cat_model = CatBoostRegressor(verbose=300,early_stopping_rounds=50, 
                              random_state=42, loss_function='RMSE', cat_features=cat_features)

In [34]:
random_cat_search = RandomizedSearchCV(estimator=cat_model, param_distributions=cat_param,
                              random_state=42, cv=2, scoring='neg_mean_squared_error')

In [35]:
%%time
random_cat_search.fit(features_train, target_train)

0:	learn: 3199.9204004	total: 445ms	remaining: 1m 28s
199:	learn: 1580.0429098	total: 1m 8s	remaining: 0us
0:	learn: 3157.8222155	total: 312ms	remaining: 1m 2s
199:	learn: 1579.8215491	total: 1m 8s	remaining: 0us
0:	learn: 4568.1006996	total: 194ms	remaining: 28.8s
149:	learn: 2691.5599639	total: 25.4s	remaining: 0us
0:	learn: 4578.9234741	total: 110ms	remaining: 16.4s
149:	learn: 2699.0646269	total: 24.9s	remaining: 0us
0:	learn: 4398.9569214	total: 51ms	remaining: 7.6s
149:	learn: 1805.8287388	total: 24.8s	remaining: 0us
0:	learn: 4402.5746311	total: 109ms	remaining: 16.3s
149:	learn: 1804.9409271	total: 25s	remaining: 0us
0:	learn: 4581.5322086	total: 800ms	remaining: 2m 39s
199:	learn: 3110.5916067	total: 2m 38s	remaining: 0us
0:	learn: 4593.1978212	total: 870ms	remaining: 2m 53s
199:	learn: 3118.0714038	total: 2m 37s	remaining: 0us
0:	learn: 4419.8850641	total: 226ms	remaining: 33.7s
149:	learn: 1636.1307087	total: 44.8s	remaining: 0us
0:	learn: 4430.3389293	total: 219ms	remaining

RandomizedSearchCV(cv=2, error_score='raise-deprecating',
                   estimator=<catboost.core.CatBoostRegressor object at 0x7fbadbf02550>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'depth': [5, 10],
                                        'iterations': [150, 200],
                                        'learning_rate': array([0.001     , 0.0011514 , 0.00132571, 0.00152642, 0.00175751,
       0.00202359, 0.00232995, 0.0026827 , 0.00308884, 0.00355648,
       0.00409492...
       0.03393222, 0.0390694 , 0.04498433, 0.05179475, 0.05963623,
       0.06866488, 0.07906043, 0.09102982, 0.10481131, 0.12067926,
       0.13894955, 0.15998587, 0.184207  , 0.21209509, 0.24420531,
       0.28117687, 0.32374575, 0.37275937, 0.42919343, 0.49417134,
       0.5689866 , 0.65512856, 0.75431201, 0.86851137, 1.        ])},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scorin

In [36]:
random_cat_search.best_params_, random_cat_search.best_score_

({'learning_rate': 0.5689866029018293, 'iterations': 200, 'depth': 5},
 -2779852.938249249)

In [37]:
%%time
mean_squared_error(target_valid, random_cat_search.predict(features_valid))**0.5

CPU times: user 498 ms, sys: 23 ms, total: 521 ms
Wall time: 450 ms


1640.0484483955545

**RandomForestRegressor**

In [38]:
sk_param_dist = {'n_estimators': [50, 70],
              'max_depth': [5, 15]}

In [39]:
sk_model = RandomForestRegressor(random_state=42)

In [40]:
sk_random_search = RandomizedSearchCV(estimator = sk_model, param_distributions=sk_param_dist, cv=2, random_state=42)

In [41]:
%%time
sk_random_search.fit(c_features_train, c_target_train)

CPU times: user 17min 20s, sys: 0 ns, total: 17min 20s
Wall time: 17min 29s


RandomizedSearchCV(cv=2, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_state=4

In [42]:
sk_random_search.best_params_, sk_random_search.best_score_

({'n_estimators': 70, 'max_depth': 15}, 0.8612370201496036)

In [43]:
%%time
mean_squared_error(c_target_valid, sk_random_search.predict(c_features_valid))**0.5

CPU times: user 1.52 s, sys: 0 ns, total: 1.52 s
Wall time: 1.53 s


1661.1754171321593

[К оглавлению](#К_оглавлению)

<a id='Анализ_моделей'></a>
# 3. Анализ моделей

In [47]:
pd.DataFrame({'LightGBMRegressor':[mean_squared_error(c_target_valid, lgbm_random_search.predict(c_features_valid))**0.5,
                                   '12min 58s', '2.49 s'], 
              'XGBoostREgressor':[mean_squared_error(c_target_valid, xgbr_random_search.predict(c_features_valid))**0.5,
                                  '22min 28s', '1.69 s'], 
              ' CatBoostRegressor':[mean_squared_error(target_valid, random_cat_search.predict(features_valid))**0.5, 
                                 '33min 45s', '528 ms'],
              'RandomForestRegressor':[mean_squared_error(c_target_valid, sk_random_search.predict(c_features_valid))**0.5,
                                     '17min 29s', '1.53 s' ]}, index=['RMSE', 'Время обучения', 'Скорость предсказания' ])

,LightGBMRegressor,XGBoostREgressor,CatBoostRegressor,RandomForestRegressor
RMSE,1615.23,1589.37,1640.05,1661.18
Время обучения,12min 58s,22min 28s,33min 45s,17min 29s
Скорость предсказания,2.49 s,1.69 s,528 ms,1.53 s


Наименьшую ошибку показала модель **XGBoostREgressor**, но если брать в расчет затраченное на обучение время, **LightGBMRegressor** показал результат лучше. Наибольшая скорость предсказания у **CatBoostRegressor**. Скорее всего в данном случае лучше всего будет использовать **XGBoostREgressor**, так как его время обучения и скорость предсказания между значениями **LightGBMRegressor** и **CatBoostRegressor**, а ошибка наименьшая. В качестве альтернативы можно выбрать **RandomForestRegressor**. Эта модель обучается быстрее чем **XGBoostREgressor**, но в то же время имеет ошибку больше.

[К оглавлению](#К_оглавлению)